# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from scipy.stats import mode
from tqdm.notebook import tqdm

In [2]:
from task import gini, entropy, gain

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
from task import DecisionTree

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [4]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1,
                 max_features="auto", n_estimators=10):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.criterion = criterion

        self.trees = []

    def fit(self, X, y):
        for i in tqdm(range(self.n_estimators)):
            tree = DecisionTree(X, y, self.criterion, self.max_depth, self.min_samples_leaf, self.max_features)
            #tree.fit()

            self.trees.append(tree)

    def predict(self, X):
        preds = self.trees[0].predict(X)

        for tree in self.trees[1:]:
            preds = np.vstack((preds, tree.predict(X)))

        return mode(preds, axis=0)[0].squeeze()

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [5]:
def feature_importance(rfc):
    err_j = np.zeros((len(rfc.trees), rfc.trees[0].oob_X.shape[1]))

    for i, tree in enumerate(rfc.trees):
        oob_X, oob_y = tree.oob_X, tree.oob_y
        N, M = oob_X.shape

        err_oob = np.mean(tree.predict(oob_X) == oob_y)        

        for j in np.arange(M):
            shuffled_X = oob_X.copy()
            random_idx = np.random.choice(np.arange(N), N, replace=False)
            shuffled_X[:, j] = oob_X[:, j][random_idx]

            err_oob_j = np.mean(tree.predict(shuffled_X) == oob_y)
            err_j[i][j] = err_oob - err_oob_j

    err_j = np.mean(err_j, axis=0)
    return err_j

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [6]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\yaros\AppData\Local\Temp\ipykernel_12416\2887499782.py:25: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(preds, axis=0)[0].squeeze()


Accuracy: 1.0
Importance: [-0.00113 -0.00851  0.20022  0.1808   0.3892  -0.00238]


### Задание 4 (3 балла)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [7]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [8]:
X, y_age, y_sex, features = read_dataset("vk_train.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [9]:
from task import rfc_age

rfc_age.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc_age.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_age), features, 20)):
    print(str(i+1) + ".", name)

  0%|          | 0/10 [00:00<?, ?it/s]

c:\Users\yaros\OneDrive\Рабочий стол\Homework 09\task.py:218: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(preds, axis=0)[0].squeeze()
c:\Users\yaros\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


Accuracy: 0.7292307692307692
Most important features:


#### Пол

In [ ]:
from task import rfc_gender

rfc_gender = RandomForestClassifier(n_estimators=10)
rfc_gender.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc_gender.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_gender), features, 20)):
    print(str(i+1) + ".", name)

  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\yaros\AppData\Local\Temp\ipykernel_16848\3653038425.py:25: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(preds, axis=0)[0].squeeze()
C:\Users\yaros\AppData\Local\Temp\ipykernel_16848\3653038425.py:25: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  return mode(preds, axis=0)[0].squeeze()


Accuracy: 0.8769230769230769
Most important features:
1. mudakoff
2. 40kg
3. 9o_6o_9o
4. rapnewrap
5. 4ch
6. girlmeme
7. academyofman
8. igm
9. zerofat
10. fuck_humor
11. bon
12. be.women
13. be.beauty
14. thesmolny
15. rhymes
16. modnailru
17. reflexia_our_feelings
18. bot_maxim
19. woman.blog
20. i_d_t


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [ ]:
X, y = synthetic_dataset(1000)

cat_params = {
                'n_estimators':500,
                'learning_rate': 0.005,
                'eval_metric':'Accuracy',
                'loss_function':'MultiClass',
                'random_seed': 42,
                'metric_period':500,
                'od_wait':500,
                'task_type':'GPU',
                'depth': 10,
                }
                
clf = CatBoostClassifier(**cat_params)
clf.fit(X, y)

y_pred = clf.predict(X).reshape(1, -1)
print("Accuracy:", np.mean(y_pred == y))
print("Importance:", clf.get_feature_importance())

0:	learn: 1.0000000	total: 19.8ms	remaining: 9.88s
499:	learn: 1.0000000	total: 8.06s	remaining: 0us
Accuracy: 1.0
Importance: [ 0.          0.          5.87260854 17.76801882 76.35937264  0.        ]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [ ]:
X, y_age, y_sex, features = read_dataset("vk_train.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [ ]:
clf = CatBoostClassifier(**cat_params)
clf.fit(X_train, y_age_train, eval_set=(X_eval, y_age_eval),
            cat_features=np.arange(X.shape[1]),
            use_best_model=True,
            verbose=True)

y_pred = clf.predict(X_test).reshape(1, -1)
clf.save_model("model_age")
print("Accuracy:", np.mean(y_pred == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 0.6452991	test: 0.6196581	best: 0.6196581 (0)	total: 32.8ms	remaining: 16.4s
499:	learn: 0.7305556	test: 0.7111111	best: 0.7111111 (498)	total: 12.3s	remaining: 0us
bestTest = 0.7111111111
bestIteration = 498
Shrink model to first 499 iterations.
Accuracy: 0.6876923076923077
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4. rhymes
5. tumblr_vacuum
6. xfilm
7. pixel_stickers
8. mudakoff
9. leprum
10. dayvinchik


#### Пол

In [ ]:
clf = CatBoostClassifier(**cat_params)
clf.fit(X_train, y_sex_train, eval_set=(X_eval, y_sex_eval),
            cat_features=np.arange(X.shape[1]),
            use_best_model=True,
            verbose=True)

y_pred = clf.predict(X_test).reshape(1, -1)
clf.save_model("model_gender")
print("Accuracy:", np.mean(y_pred == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 0.7512821	test: 0.7316239	best: 0.7316239 (0)	total: 27.6ms	remaining: 2m 17s
500:	learn: 0.8623932	test: 0.8743590	best: 0.8743590 (500)	total: 9.98s	remaining: 1m 29s
1000:	learn: 0.8841880	test: 0.8820513	best: 0.8829060 (977)	total: 20.3s	remaining: 1m 21s
1500:	learn: 0.8961538	test: 0.8846154	best: 0.8854701 (1470)	total: 30.5s	remaining: 1m 11s
2000:	learn: 0.9047009	test: 0.8871795	best: 0.8888889 (1764)	total: 40.9s	remaining: 1m 1s
bestTest = 0.8888888889
bestIteration = 1764
Shrink model to first 1765 iterations.
Accuracy: 0.8553846153846154
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. thesmolny
6. 9o_6o_9o
7. zerofat
8. i_d_t
9. reflexia_our_feelings
10. academyofman


In [ ]:
from task import catboost_rfc_age, catboost_rfc_gender
